In [ ]:
!pip install streamlit groq pyngrok -q


In [ ]:
with open('mental_health_app.py', 'w') as f:
    f.write('''import streamlit as st
from groq import Groq
import datetime

# Initialize Groq client
client = Groq(api_key="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")

# Page configuration
st.set_page_config(page_title="Mental Health Companion", page_icon="🌸", layout="wide")

# Custom CSS
st.markdown("""
<style>
    .stApp {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    }
    .chat-message {
        padding: 1.5rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
        display: flex;
        flex-direction: column;
    }
    .user-message {
        background-color: rgba(255, 255, 255, 0.1);
        margin-left: 20%;
    }
    .bot-message {
        background-color: rgba(255, 255, 255, 0.2);
        margin-right: 20%;
    }
</style>
""", unsafe_allow_html=True)

# Initialize session state
if 'messages' not in st.session_state:
    st.session_state.messages = []
if 'mood_log' not in st.session_state:
    st.session_state.mood_log = []

# System prompt for the AI
SYSTEM_PROMPT = """You are a compassionate and empathetic mental health companion. Your role is to:
- Listen actively and provide emotional support
- Offer coping strategies and mindfulness techniques
- Encourage positive thinking and self-care
- Validate feelings without judgment
- Suggest professional help when needed
- Be warm, understanding, and supportive

Remember: You are NOT a replacement for professional mental health care. Always encourage users to seek professional help for serious concerns."""

def get_ai_response(user_message, conversation_history):
    """Get response from Groq AI"""
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    # Add conversation history
    for msg in conversation_history[-6:]:  # Last 3 exchanges
        messages.append(msg)

    messages.append({"role": "user", "content": user_message})

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages,
        max_tokens=500,
        temperature=0.7
    )

    return response.choices[0].message.content

# Header
st.markdown("<h1 style='text-align:center; color:white; font-size:48px;'>🌸 Mental Health Companion</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align:center; color:white; font-size:18px;'>Your safe space for emotional support and wellness</p>", unsafe_allow_html=True)

# Sidebar
with st.sidebar:
    st.markdown("### 🌟 Quick Tools")

    # Mood tracker
    st.markdown("#### How are you feeling today?")
    mood = st.select_slider(
        "",
        options=["😢 Very Bad", "😞 Bad", "😐 Okay", "🙂 Good", "😊 Great"],
        value="😐 Okay"
    )

    if st.button("Log Mood"):
        st.session_state.mood_log.append({
            'mood': mood,
            'time': datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        })
        st.success("Mood logged! 💚")

    st.markdown("---")

    # Quick exercises
    st.markdown("#### 🧘 Quick Exercises")
    if st.button("Breathing Exercise"):
        st.info("**Box Breathing**\\n\\n1. Breathe in for 4 seconds\\n2. Hold for 4 seconds\\n3. Breathe out for 4 seconds\\n4. Hold for 4 seconds\\n\\nRepeat 4 times 🌬️")

    if st.button("Grounding Technique"):
        st.info("**5-4-3-2-1 Technique**\\n\\nName:\\n- 5 things you can see\\n- 4 things you can touch\\n- 3 things you can hear\\n- 2 things you can smell\\n- 1 thing you can taste")

    if st.button("Positive Affirmation"):
        import random
        affirmations = [
            "You are worthy of love and respect 💜",
            "You are doing better than you think 🌟",
            "It's okay to take things one day at a time 🌅",
            "Your feelings are valid 💙",
            "You have the strength to get through this 💪",
            "Be kind to yourself today 🌸"
        ]
        st.success(random.choice(affirmations))

    st.markdown("---")

    # Resources
    st.markdown("#### 📞 Crisis Resources")
    st.markdown("""
    - **National Crisis Line**: 988
    - **Crisis Text Line**: Text HOME to 741741
    - **International**: findahelpline.com
    """)

    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.rerun()

# Main chat area
st.markdown("### 💬 Chat")

# Display chat history
for message in st.session_state.messages:
    if message["role"] == "user":
        st.markdown(f"""
        <div class="chat-message user-message">
            <strong>You:</strong><br>{message["content"]}
        </div>
        """, unsafe_allow_html=True)
    else:
        st.markdown(f"""
        <div class="chat-message bot-message">
            <strong>🌸 Companion:</strong><br>{message["content"]}
        </div>
        """, unsafe_allow_html=True)

# Chat input
user_input = st.chat_input("Share what's on your mind... I'm here to listen 💙")

if user_input:
    # Add user message
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Get AI response
    with st.spinner("Thinking..."):
        ai_response = get_ai_response(user_input, st.session_state.messages)

    # Add AI response
    st.session_state.messages.append({"role": "assistant", "content": ai_response})

    # Rerun to update chat
    st.rerun()

# Welcome message if chat is empty
if len(st.session_state.messages) == 0:
    st.markdown("""
    <div style='background-color: rgba(255,255,255,0.2); padding: 2rem; border-radius: 1rem; margin-top: 2rem;'>
        <h3 style='color: white;'>👋 Welcome! I'm here for you.</h3>
        <p style='color: white;'>This is a safe, judgment-free space where you can share your thoughts and feelings.
        Whether you're having a tough day or just need someone to talk to, I'm here to listen and support you.</p>
        <p style='color: white;'><strong>You can talk to me about:</strong></p>
        <ul style='color: white;'>
            <li>Stress and anxiety</li>
            <li>Mood and emotions</li>
            <li>Coping strategies</li>
            <li>Self-care tips</li>
            <li>Or anything else on your mind</li>
        </ul>
        <p style='color: white; font-size: 14px;'><em>⚠️ Important: I'm here to support you, but I'm not a replacement for professional mental health care.
        If you're in crisis or need immediate help, please reach out to a crisis helpline.</em></p>
    </div>
    """, unsafe_allow_html=True)
''')

print("✅ App file created!")

✅ App file created!


In [ ]:
import subprocess
from pyngrok import ngrok
import time

print("🚀 Starting Streamlit server...")

# Start streamlit in background
subprocess.Popen(['streamlit', 'run', 'mental_health_app.py', '--server.headless=true'],
                 stdout=subprocess.DEVNULL,
                 stderr=subprocess.DEVNULL)

# Wait for server to start
time.sleep(5)

print("🌐 Creating public URL...")

# Create tunnel
public_url = ngrok.connect(8501)

print("\n" + "="*60)
print("✨ YOUR MENTAL HEALTH COMPANION IS READY! ✨")
print("="*60)
print(f"\n🔗 Click here to open: {public_url}\n")
print("💡 Tips:")
print("   • Keep this Colab tab open while using the app")
print("   • The URL will change if you restart Colab")
print("   • Session lasts ~12 hours on free tier")
print("\n" + "="*60)

🚀 Starting Streamlit server...
🌐 Creating public URL...

✨ YOUR MENTAL HEALTH COMPANION IS READY! ✨

🔗 Click here to open: NgrokTunnel: "https://dutiful-elda-conceptualistic.ngrok-free.dev" -> "http://localhost:8501"

💡 Tips:
   • Keep this Colab tab open while using the app
   • The URL will change if you restart Colab
   • Session lasts ~12 hours on free tier

